In [14]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch.nn import LSTM, GRU, LSTMCell

In [15]:
import pandas as pd

# 创建包含数据的字典
data = {
    'Date': ['2017-01-04', '2017-01-05', '2017-01-06', '2017-01-10', '2017-01-11'],
    'Close': [2742.0, 2738.0, 2740.0, 2748.0, 2745.0]
}

# 使用字典创建Pandas数据框
df = pd.DataFrame(data)
df = pd.concat([df]*100,axis=0)
# 打印输出整理后的数据框
print(df.head(2))

         Date   Close
0  2017-01-04  2742.0
1  2017-01-05  2738.0


In [16]:
# 数据准备
time_data = df[['Close']].values.astype('float32')
train_size = int(len(time_data) * 0.67)
test_size = len(time_data) - train_size
train, test = time_data[:train_size], time_data[train_size:]
# 转为tensor[batch_size, seq_len, emb_size]
train_tensor = torch.FloatTensor(train).view(-1, train.shape[0], 1)
test_tensor = torch.FloatTensor(test).view(-1, test.shape[0], 1)

In [22]:
test_tensor.shape

torch.Size([1, 165, 1])

In [95]:
# 定义网路架构LSTM
# input_size   hidden_size  num_layers out_size
class LstmModel(nn.Module):
    def __init__(self,input_size=1, hidden_size=50, num_layers=1, out_size=1, bidirectional=False, batch_first=True):
        super().__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = 2 if bidirectional else 1
        self.lstm = LSTM(input_size,hidden_size,num_layers,batch_first=batch_first, bidirectional=bidirectional)
        self.fc = nn.Linear(self.bidirectional*hidden_size, out_size)

    def forward(self,x):
        h0 = torch.randn(self.bidirectional*self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        c0 = torch.randn(self.bidirectional*self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        output, (_, _) = self.lstm(x, (h0.detach(), c0.detach()))
        # output = output.contiguous().view(x.size(0), x.size(1), 2, self.hidden_size)
        # output = torch.mean(output,dim=2)
        # print(output.size())
        return self.fc(output[:, :, :])

In [96]:
model = LstmModel()
model(train_tensor).size()

torch.Size([1, 335, 1])

In [97]:
# 超参数设置
input_size = 1
hidden_size = 100
num_layers = 5
output_size = 1
learning_rate = 0.1
num_epochs = 1500

# 实例化模型
model = LstmModel(input_size, hidden_size, num_layers, output_size)

# 定义损失函数与优化算法
criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()
# 开始进行训练
for epoch in range(num_epochs):
    outputs = model(train_tensor)
    optimizer.zero_grad()
    loss = criterion(outputs, train_tensor[:, :, :])
    loss.backward()
    optimizer.step()

    if (epoch+1) % 50 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("训练完成")

Epoch [50/1500], Loss: 6301422.5000
Epoch [100/1500], Loss: 5200031.5000
Epoch [150/1500], Loss: 4254151.5000
Epoch [200/1500], Loss: 3449186.5000
Epoch [250/1500], Loss: 2766157.2500
Epoch [300/1500], Loss: 2197035.0000
Epoch [350/1500], Loss: 1722692.7500
Epoch [400/1500], Loss: 1336736.6250
Epoch [450/1500], Loss: 1022974.2500
Epoch [500/1500], Loss: 772660.8750
Epoch [550/1500], Loss: 574936.6250
Epoch [600/1500], Loss: 423565.1250
Epoch [650/1500], Loss: 288541.9062
Epoch [700/1500], Loss: 204283.7969
Epoch [750/1500], Loss: 142855.6719
Epoch [800/1500], Loss: 98134.4453
Epoch [850/1500], Loss: 66346.0469
Epoch [900/1500], Loss: 45789.6406
Epoch [950/1500], Loss: 29004.1309
Epoch [1000/1500], Loss: 20190.9453
Epoch [1050/1500], Loss: 12372.7256
Epoch [1100/1500], Loss: 9620.9795
Epoch [1150/1500], Loss: 7874.3281
Epoch [1200/1500], Loss: 5085.8472
Epoch [1250/1500], Loss: 2980.0425
Epoch [1300/1500], Loss: 3778.1970
Epoch [1350/1500], Loss: 2363.4355
Epoch [1400/1500], Loss: 2380.

In [99]:
# 模型验证
def MSE(Y_ture,Y_predict):
    return ((Y_ture - Y_predict)**2).sum()/Y_ture.shape[0]
model.eval()
test_outputs = model(test_tensor).detach().numpy()
print("平均：",MSE(train,train.mean()),MSE(test,test.mean()))
MSE(test,test_outputs)

4862.555681818182

In [ ]:
# 构建双层LSTMcell
# input_size, hidden_size, num_layers, output_size
class LstmCellModel(nn.module):
    def __init__(self, input_size=1, hidden_size1=100, hidden_size2=50, output=1, batch_first=True):


    def forward(self,x):
        pass